### <Q1, Q2를 추정하기 위한 이중 Q 학습> 
출처 : 단단한 머신러닝 챕터 6 

알고리즘 파라미터 : 시간 간격 $\alpha \in (0,1] $, 작은 양수 $\epsilon > 0$ 

모든 $s \in S^+$ 에 대한 Q1(s,a), Q2(s,a)를 임의의 값으로 초기화. 단, Q(종단, -) = 0 

각 에피소드에 대한 루프 : 
- S를 초기화 
- 에피소드의 각 단계에 대한 루프 : 

> Q1 +Q2 에 있어서 입실론 탐욕적인 정책을 사용하여 S'으로부터 A'를 선택 

> 행동 A를 취하고 $R,S'$ 을 관측 

> 0.5의 확률로:
  >Q1(s,a) ← $Q_1(s,a) + \alpha[R+ \gamma  Q_2(s',argmax_a Q_1(S',a)) - Q_1(s,a)] $

> 그 밖의 경우 : 
  >Q2(s,a) ← $Q_2(s,a) + \alpha[R+ \gamma Q_1(s',argmax_a Q_2(S',a)) - Q_2(s,a)] $ 

> S ← S', A ← A' 

S가 종단이면 종료 


**<구현해야 하는 것>**
- Q1(s,a) : 
- Q2(s,a) : 
- 입실론 탐욕적인 정책 


**<필요한 것>** 
- $\alpha$ : class 제작시 입력 값으로 부여 
- $\epsilon$ : 충분히 작은 값으로 

**<함수 / 데이터 형식>** 
- class evaluate_TD : # alpha 값 추가 
> def __init__(self, S, A, alpha reward_func, epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 


**<외부함수>** 
- R(s',a,s) : 보상함수. 챕터 4의 코드 참고 
- choose_random_max(lst) : lst 중 가장 값이 큰 것을 반환. 혹시 max 값이 중복된다면 임의의 하나 산출

**<고민점>** 
- Q1 + Q2 를 어떻게 표현해야 하는가? 

- $pi(s,a) = \pi(a|s)$ 를 표현할 데이터 형식 필요. 갱신을 위해서는 함수 형태가 아니라 데이터 형식이 필요함 

> 이중 리스트, dict가 있음. 이 중에서 dict 사용 

- pi(s,a)를 언제마다 갱신할 것인가? 

> update_return 함수 시작 부분에 하면 매번 상호 영향을 줄 수 있을 듯 

In [1]:
# 테스트 용 임시 데이터 
S = list(range(100)) 
A = list(range(10))

In [2]:
from collections import defaultdict
import random
import numpy as np

In [7]:
# 외부용 함수 reward_func 간략 구현 
def reward_func(next_s, a, s) : 
    # next_s 와 s의 차이가 짝수이면 +1, 홀수면 -1 
    if abs(next_s - s) %2 == 0 and next_s > s : reward = 1 
    else : reward = -1
    
    return reward 
    
#최대값이 2개 이상인 경우, 임의로 1개의 최대값을 만들어낸 행동 a를 산출 
def choose_random_max(lst) :
    max_arg = np.where(np.array(lst) >= max(lst))
    return random.choice(list(max_arg)[0]) #max_arg가 array 형태로 안에 있는 list를 꺼내기 위해 [0] 사용 



In [27]:
# evaluate_Q_TD 코드 참고
# 마지막 Q(s,a) 갱신 부분만 변경 

class evaluate_double_Q :  
    def __init__(self, S, A, reward_func, alpha=0.1,  epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 
        self.S = S 
        self.A = A 
        self.reward_func = reward_func
        self.alpha = alpha
        self.epsilon = epsilon 
        self.gamma = gamma 
        self.num_episode = num_episode
        self.T = len_episode
        self.Q1, self.Q2 = self.initiate_Q(), self.initiate_Q() 
        self.pi = self.initiate_pi() 

#        self.b, self.C = self.initiate_b()
#        self.V = self.initiate_V() 
    
    def initiate_Q(self) : # Q(s,a) 값을 초기화 
        Q_dict = defaultdict(float)
        for s in self.S : 
            for a in self.A : 
                Q_dict[(s,a)] = 0         
        
        return Q_dict
    
    def initiate_pi(self) : # pi(s,a) 값을 초기화 
        pi_dict = defaultdict(float)
        for s in self.S : 
            for a in self.A : 
                pi_dict[(s,a)] = 1/len(self.A)         
        
        return pi_dict
    
    def update_pi(self, s) : # 입실론 탐욕적 정책에 맞게 값 변경. 확률값 list로 반환할 것. 
        lst = [] 
        opt_a = choose_random_max([self.Q1[(s,a)]+ self.Q2[(s,a)] for a in self.A])
        for a in self.A : 
            if a == opt_a : self.pi[(s,a)] = (1-self.epsilon) + (1/self.epsilon)/len(self.A) 
            else : self.pi[(s,a)] = (1/self.epsilon)/len(self.A)
                

    def choice_action(self, s, policy) : #일반화. next_s 까지 반환하도록 수정 
        policy_a_list = [] 
        for _ in self.A :
            policy_a_list.append(policy[(s,_)])  
        a= random.choices(self.A, weights = policy_a_list)
        a = a[0]
        
        return a
    
    def next_s(self, s,a) : # 상태 s에서 a 행동을 했을 때 다음 상태 s'. 정책, S,A 에 따라 달라짐. 
        return min(max(s+a, 0), max(self.S)) 

    def make_episode(self, start_s, T) :
        s = start_s
        episode = {"S" : [], "A" : [], "R" : []}
        episode["R"].append(0) # R_0 값 부여 
        for _ in range(T) : 
            episode["S"].append(s)
            a = self.choice_action(s, self.pi)   
            next_s = self.next_s(s, a)
            r = self.reward_func(next_s, a, s)
            episode["A"].append(a)
            episode["R"].append(r)
            s = next_s
        return episode 
 
    def update_returns(self) : # Q 추정 및 제어를 위해 수정 
        for s in self.S : self.update_pi(s)
        
        for _ in range(self.num_episode) : 
            start_s = random.choice(self.S) # 시작 탐험 가정 
            episode = self.make_episode(start_s, self.T) 
            S,R,A = episode['S'],episode['R'], episode['A'] 
            # make_episode 에서 이미 a,r를 계산해 두었기 때문에 Q(s,a)만 갱신하겠음. 
            rand = random.choice([0,1])
                
            for index, s in enumerate(S[:-1]) :  
                index_origin_s = self.S.index(s)
                next_s = S[index+1]
                if rand == 0 : 
                    max_a = self.A[np.argmax([self.Q1[(next_s,a)] for a in self.A])] # 행동 값으로 나와야 함. 
                    self.Q1[(s, R[index])] = self.Q1[(s,R[index])] + self.alpha*(R[index+1] + self.gamma*self.Q2[(next_s, max_a)] - self.Q1[(s, R[index])]) 
                    
                else : 
                    max_a = self.A[np.argmax([self.Q2[(next_s,a)] for a in self.A])] # 행동 값으로 나와야 함. 
                    self.Q2[(s, R[index])] = self.Q2[(s,R[index])] + self.alpha*(R[index+1] + self.gamma*self.Q1[(next_s, max_a)] - self.Q2[(s, R[index])]) 
                    


In [28]:
test = evaluate_double_Q(S,A, reward_func)
lst = [] 

for _ in range(100) : 
    m_lst = [] 
    for s in S :
        q1_lst = [test.Q1[(s,a)] for a in A]
        q2_lst = [test.Q2[(s,a)] for a in A]

        max_a = A[q1_lst.index(max(q1_lst))]
        max_b = A[q2_lst.index(max(q2_lst))]
        m_lst.append([max_a, max_b])
    lst.append(m_lst)
    test.update_returns()

print(lst[0])
print(lst[-1])



[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[1, 1], [1, 1], [1, 1], [1, 2], [2, 1], [1, 1], [2, 0], [1, 2], [1, 0], [1, 0], [0, 1], [0, 0], [0, 0], [0, 2], [2, 2], [0, 2], [0, 2], [0, 1], [1, 2], [0, 0], [0, 2], [2, 2], [0, 2], [2, 2], [1, 0]

In [26]:
rand = random.choice([0,1])
print(rand)

1
